<a href="https://colab.research.google.com/github/lynn-e/ml-guide/blob/master/ch08/8_7_%EB%AC%B8%EC%84%9C_%EA%B5%B0%EC%A7%91%ED%99%94_%EC%86%8C%EA%B0%9C%EC%99%80_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

In [ ]:
import pandas as pd
import glob, os 

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
all_files = glob.glob('/content/gdrive/My Drive/notebook/ml-guide/data/OpinosisDataset1.0/topics/*.data')
filename_list = []
opinion_text = []

# 개별 파일의 파일명은 filename_list로 취합
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame 생성
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')

  # 파일명 가공 및 .data 확장자 제거
  filename_= file_.split('/')[-1]
  filename = filename_.split('.')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가.
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename': filename_list, 'opinion_text': opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,display_garmin_nuvi_255W_gps,...
2,directions_garmin_nuvi_255W_gps,...
3,keyboard_netbook_1005ha,...
4,speed_windows7,...


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85)
# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 군집 label 추가 
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,4
1,display_garmin_nuvi_255W_gps,...,0
2,directions_garmin_nuvi_255W_gps,...,0
3,keyboard_netbook_1005ha,...,4
4,speed_windows7,...,2


In [ ]:
# 군집 결과 확인 (cluster_label==0)
document_df[document_df['cluster_label'] == 0].sort_values(by='filename')# 군집 결과 확인 (cluster_label==0)
document_df[document_df['cluster_label'] == 0].sort_values(by='filename')

,filename,opinion_text,cluster_label
15,accuracy_garmin_nuvi_255W_gps,...,0
2,directions_garmin_nuvi_255W_gps,...,0
1,display_garmin_nuvi_255W_gps,...,0
11,satellite_garmin_nuvi_255W_gps,...,0
6,screen_garmin_nuvi_255W_gps,...,0
16,speed_garmin_nuvi_255W_gps,...,0
10,updates_garmin_nuvi_255W_gps,...,0
17,voice_garmin_nuvi_255W_gps,...,0


In [ ]:
# 군집 결과 확인 (cluster_label==1)
document_df[document_df['cluster_label'] == 1].sort_values(by='filename')

,filename,opinion_text,cluster_label
41,bathroom_bestwestern_hotel_sfo,...,1
25,food_holiday_inn_london,...,1
34,food_swissotel_chicago,...,1
42,free_bestwestern_hotel_sfo,...,1
46,location_bestwestern_hotel_sfo,...,1
27,location_holiday_inn_london,...,1
37,parking_bestwestern_hotel_sfo,...,1
19,price_holiday_inn_london,...,1
21,room_holiday_inn_london,...,1
40,rooms_bestwestern_hotel_sfo,...,1


In [ ]:
# 군집 결과 확인 (cluster_label==2)
document_df[document_df['cluster_label'] == 2].sort_values(by='filename')

,filename,opinion_text,cluster_label
49,comfort_honda_accord_2008,...,2
31,comfort_toyota_camry_2007,...,2
14,features_windows7,...,2
20,gas_mileage_toyota_camry_2007,...,2
38,interior_honda_accord_2008,...,2
22,interior_toyota_camry_2007,...,2
39,mileage_honda_accord_2008,...,2
47,performance_honda_accord_2008,...,2
44,quality_toyota_camry_2007,...,2
43,seats_honda_accord_2008,...,2


In [ ]:
from sklearn.cluster import KMeans

# 3개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 군집 label 추가 
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
1,display_garmin_nuvi_255W_gps,...,0
2,directions_garmin_nuvi_255W_gps,...,0
3,keyboard_netbook_1005ha,...,0
4,speed_windows7,...,0


In [ ]:
# 군집 결과 확인 (cluster_label==0)
document_df[document_df['cluster_label'] == 0].sort_values(by='filename')

,filename,opinion_text,cluster_label
15,accuracy_garmin_nuvi_255W_gps,...,0
26,battery-life_amazon_kindle,...,0
0,battery-life_ipod_nano_8gb,...,0
9,battery-life_netbook_1005ha,...,0
35,buttons_amazon_kindle,...,0
2,directions_garmin_nuvi_255W_gps,...,0
1,display_garmin_nuvi_255W_gps,...,0
32,eyesight-issues_amazon_kindle,...,0
14,features_windows7,...,0
18,fonts_amazon_kindle,...,0


In [ ]:
# 군집 결과 확인 (cluster_label==1)
document_df[document_df['cluster_label'] == 1].sort_values(by='filename')

,filename,opinion_text,cluster_label
41,bathroom_bestwestern_hotel_sfo,...,1
25,food_holiday_inn_london,...,1
34,food_swissotel_chicago,...,1
42,free_bestwestern_hotel_sfo,...,1
46,location_bestwestern_hotel_sfo,...,1
27,location_holiday_inn_london,...,1
37,parking_bestwestern_hotel_sfo,...,1
19,price_holiday_inn_london,...,1
21,room_holiday_inn_london,...,1
40,rooms_bestwestern_hotel_sfo,...,1


In [ ]:
# 군집 결과 확인 (cluster_label==2)
document_df[document_df['cluster_label'] == 2].sort_values(by='filename')

,filename,opinion_text,cluster_label
49,comfort_honda_accord_2008,...,2
31,comfort_toyota_camry_2007,...,2
20,gas_mileage_toyota_camry_2007,...,2
38,interior_honda_accord_2008,...,2
22,interior_toyota_camry_2007,...,2
39,mileage_honda_accord_2008,...,2
47,performance_honda_accord_2008,...,2
44,quality_toyota_camry_2007,...,2
43,seats_honda_accord_2008,...,2
48,transmission_toyota_camry_2007,...,2


### 군집별 핵심 단어 추출하기

In [ ]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


In [ ]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num, top_n_features=10):
  cluster_details = {}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(cluster_num):
    # 개별 군집별 정보를 담을 데이터 초기화.
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함.
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']  
    filenames = filenames.values.tolist()

    cluster_details[cluster_num]['filenames'] = filenames

  return cluster_details

In [ ]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features: ', cluster_detail['top_features'])
    print('Reviews 파일명 : ', cluster_detail['filenames'][:7])
    print('==============================================')

In [ ]:
feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, cluster_num=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features:  ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 :  ['battery-life_ipod_nano_8gb', 'display_garmin_nuvi_255W_gps', 'directions_garmin_nuvi_255W_gps', 'keyboard_netbook_1005ha', 'speed_windows7', 'video_ipod_nano_8gb', 'screen_garmin_nuvi_255W_gps']
###### Cluster 1
Top features:  ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 :  ['price_holiday_inn_london', 'room_holiday_inn_london', 'staff_swissotel_chicago', 'service_holiday_inn_london', 'food_holiday_inn_london', 'location_holiday_inn_london', 'rooms_swissotel_chicago']
###### Cluster 2
Top features:  ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 :  ['gas_mileage_toyota_camry_2007', 'interior_toyota_camry_2007', 'comfort_toyota_camry_2007', 'interior_honda_accord_2008', 'mileage_honda_ac